In [1]:
import pandas as pd
import os

In [2]:
# Load the dataset
file_path = '/root/juno/lateralraise_coords_personid.csv'
data = pd.read_csv(file_path)

In [3]:
# Output directory for saved time-series files
output_dir = '/root/juno/time_series_data'
os.makedirs(output_dir, exist_ok=True)

# Extract unique keypoint combinations
keypoint_combinations = data[['class', 'posture_type', 'person_id']].drop_duplicates()

# Initialize a list to hold processed data
all_time_series_data = []

In [4]:
# Iterate through each keypoint combination
for _, row in keypoint_combinations.iterrows():
    class_label = row['class']
    posture_type = row['posture_type']
    person_id = row['person_id']
    
    # Filter data for the current keypoint combination
    subset = data[(data['class'] == class_label) & 
                  (data['posture_type'] == posture_type) & 
                  (data['person_id'] == person_id)]
    
    # Prepare time-series data
    time_series_dict = {}
    for coord in ['x', 'y', 'z']:
        coord_columns = [col for col in subset.columns if col.startswith(coord)]
        time_series_dict[coord] = subset[coord_columns].values.tolist()
    
    # Append label and time-series data
    time_series_entry = {
        'label': class_label,
        'posture_type': posture_type,
        'person_id': person_id,
        'x_series': time_series_dict['x'],
        'y_series': time_series_dict['y'],
        'z_series': time_series_dict['z']
    }
    all_time_series_data.append(time_series_entry)

    # Save each keypoint combination to an individual file (optional)
    subset_file = os.path.join(output_dir, f'{class_label}_{posture_type}_{person_id}.csv')
    subset.to_csv(subset_file, index=False)



In [5]:
# Save the entire dataset as a single file
time_series_df = pd.DataFrame(all_time_series_data)
time_series_df.to_csv(os.path.join(output_dir, 'multivariate_time_series.csv'), index=False)

print("Time-Series Data saved successfully.")

Time-Series Data saved successfully.
